In [1]:
#Import Pandas and Geopandas for spatial analysis
import pandas as pd
import geopandas as gpd
import folium
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Read Excel coordinates
coords = pd.read_csv("/content/drive/MyDrive/Location in Nigeria.csv")


In [3]:
coords.head()

,Latitude,Longitude
0,9.419985,4.103187
1,13.000490,12.450214
2,12.818884,6.074024
3,11.820470,6.803141
4,13.912474,3.589053


In [4]:
coords.columns


Index(['Latitude', 'Longitude'], dtype='object')

In [5]:
# Convert to Geopandas points layer
points = gpd.GeoDataFrame(coords, geometry=gpd.points_from_xy(coords.Longitude, coords.Latitude))



In [6]:
points.head()

,Latitude,Longitude,geometry
0,9.419985,4.103187,POINT (4.10319 9.41998)
1,13.000490,12.450214,POINT (12.45021 13.00049)
2,12.818884,6.074024,POINT (6.07402 12.81888)
3,11.820470,6.803141,POINT (6.80314 11.82047)
4,13.912474,3.589053,POINT (3.58905 13.91247)


In [8]:
# Read wards shapefile
wards = gpd.read_file("/content/drive/MyDrive/Nig Wards/Nigeria_-_Ward_Boundaries.shp")


In [9]:
wards.head(5)

,FID,globalid,uniq_id,timestamp,editor,wardname,wardcode,lganame,lgacode,statename,statecode,amapcode,status,source,urban,Shape__Are,Shape__Len,geometry
0,1,564c8211-f688-4091-b45e-a2862ddcfb54,499246,2020-08-31,najib.adam,Ikuru,RVSNDN06,Andoni,33005,Rivers,RI,NIE RVS NDN,Operational Placeholder,INEC,No,0.001024,0.144684,"POLYGON ((7.39065 4.43648, 7.38503 4.4791, 7.3..."
1,2,a3a0fa0f-b22c-4085-9bdb-1e1c3473d79a,499247,2019-03-16,oluseye.abraham,Aktward 17,RVSABM17,Akuku Toru,33004,Rivers,RI,NIE RVS ABM,Operational Placeholder,INEC,No,0.004085,0.310204,"POLYGON ((6.71866 4.61093, 6.71477 4.6205, 6.7..."
2,3,d89b1914-78a0-4003-9bb0-8f3ab6c83517,499249,2019-03-16,oluseye.abraham,Aktward 1,RVSABM01,Akuku Toru,33004,Rivers,RI,NIE RVS ABM,Operational Placeholder,INEC,No,0.009486,0.399721,"POLYGON ((6.82083 4.63792, 6.83814 4.61875, 6...."
3,4,ef78ea95-08ac-4eac-a66d-64c58f9c4e15,499251,2019-03-16,oluseye.abraham,Ward 12,CRSANA12,Calabar South,9009,Cross River,CR,None,Operational Placeholder,INEC,No,0.008935,0.447254,"POLYGON ((8.30619 4.98152, 8.30968 4.97872, 8...."
4,5,12b3ac63-95d7-4256-9cb4-1c11905a642b,499252,2019-03-16,oluseye.abraham,Koluama,BYSSPR09,Southern Ijaw,6007,Bayelsa,BY,NIE BYS SPR,Operational Placeholder,INEC,No,0.026512,0.810236,"POLYGON ((5.98169 4.42706, 5.97855 4.41279, 5...."


In [9]:
#Project the points to the same CRS as the wards shapefile
points = points.set_crs(epsg=4326)
points_reprojected = points.to_crs(epsg=4326)


In [27]:
# Create a map
m = folium.Map(location=[6.5243, 3.3792], zoom_start=10)

# Convert 'points' to GeoJSON
points_geojson = points[['geometry']].to_crs(epsg=4326).__geo_interface__

# Convert 'points' to GeoJSON
points_geojson = points[['geometry']].to_crs(epsg=4326).__geo_interface__
for feature in points_geojson['features']:
    coords = feature['geometry']['coordinates']
    folium.CircleMarker(
        location=coords,
        radius=1,
        fill_color='blue',
        color='blue'
    ).add_to(m)


# Display the map
m

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Add wards to the map (assuming 'wards' is already loaded)
# Convert timestamp columns to strings
for col in wards.select_dtypes(include=['datetime64']).columns:
    wards[col] = wards[col].astype(str)

folium.GeoJson(wards).add_to(m)
m

Buffered data was truncated after reaching the output size limit.

In [10]:
#Perform spatial join where point fall within the ward
joined = gpd.sjoin(points, wards, predicate='within')

In [11]:
#check if point and ward are in thesame projection
print(points.crs)
print(wards.crs)


EPSG:4326
EPSG:4326


In [15]:
joined.head()

,Latitude,Longitude,geometry,index_right,FID,globalid,uniq_id,timestamp,editor,wardname,...,lganame,lgacode,statename,statecode,amapcode,status,source,urban,Shape__Are,Shape__Len
0,9.419985,4.103187,POINT (4.10319 9.41998),6336,6337,01269224-a7b9-4d7c-b77a-7df1dd7a2a07,493539,2019-03-16,oluseye.abraham,Gwaria,...,Kaiama,24011,Kwara,KW,NIE KWS KMA,Operational Validated,INEC,No,0.078550,2.154228
1,13.000490,12.450214,POINT (12.45021 13.00049),9114,9115,74108a21-cacf-40a2-8363-e2e7563e7afe,495620,2019-03-16,oluseye.abraham,Zanna Umarti,...,Mobbar,122,Borno,BR,NIE BOS MBR ZUU,Operational Validated,INEC,No,0.010345,0.505844
2,12.818884,6.074024,POINT (6.07402 12.81888),4112,4113,3223bc12-1d2a-477a-893d-ee065991ee60,496958,2019-03-16,oluseye.abraham,Janbako,...,Maradun,809,Zamfara,ZA,NIE ZAS MRD JAN,Operational Validated,INEC,No,0.027925,1.066561
3,11.820470,6.803141,POINT (6.80314 11.82047),5150,5151,40d9abaf-3719-4f64-98dd-5a02f3ebee7a,492297,2019-03-16,oluseye.abraham,Keta,...,Tsafe,813,Zamfara,ZA,NIE ZAS TSF KET,Operational Validated,INEC,No,0.013714,0.694960
5,11.753653,6.189018,POINT (6.18902 11.75365),4134,4135,472516ba-91ec-4030-b0da-a4aacf7d151f,496987,2019-03-16,oluseye.abraham,Sabon Birni,...,Anka,801,Zamfara,ZA,NIE KNS GRZ SAB,Operational Validated,INEC,No,0.054168,1.276243


In [14]:
# Add ward name to original Excel sheet
coords['wardname'] = joined['wardname']  # Assuming 'WARD_NAME' is the column name in wards shapefile



In [15]:
coords['wardname'].head(15)

,wardname
0,Gwaria
1,Zanna Umarti
2,Janbako
3,Keta
4,NaN
5,Sabon Birni
6,Agalegu 2
7,Bilkicheri
8,Mutai
9,Dazara


In [16]:
# Export to Excel
coords.to_excel("Wards_in_Nigeria.xlsx", index=False)
